# 준비

In [133]:
import numpy as np
import pandas as pd

train = pd.read_csv("data/FIFA_train.csv")
test = pd.read_csv("data/FIFA_test.csv")

# 전처리

In [134]:
train["log_value"] = np.log(train["value"])
all_data = pd.concat([train, test], ignore_index=True)

In [135]:
from sklearn.decomposition import PCA

pca = PCA(n_components=1)
all_data["all_stat"] =pca.fit_transform(all_data[ ["stat_overall", "stat_potential"]])
all_data = all_data.drop(["stat_overall", "stat_potential"], axis=1)

In [136]:
all_data.loc[all_data["contract_until"]=="Jun 30, 2019","contract_until"] = 2018
all_data.loc[all_data["contract_until"]=="Dec 31, 2018","contract_until"] = 2018
all_data.loc[all_data["contract_until"]=="May 31, 2019","contract_until"] = 2019
all_data.loc[all_data["contract_until"]=="Jan 31, 2019","contract_until"] = 2018
all_data.loc[all_data["contract_until"]=="Jun 30, 2020","contract_until"] = 2019
all_data.loc[all_data["contract_until"]=="Jan 1, 2019","contract_until"] = 2018
all_data.loc[all_data["contract_until"]=="May 31, 2020","contract_until"] = 2020
all_data.loc[all_data["contract_until"]=="Jan 12, 2019","contract_until"] = 2018
all_data.loc[all_data["contract_until"]=="Dec 31, 2019","contract_until"] = 2020
all_data.loc[all_data["contract_until"]=="Jun 1, 2019","contract_until"] = 2019

all_data["contract_until"] = all_data["contract_until"].astype(int)
all_data["contract_until"] = all_data["contract_until"].apply(lambda x: x-2018)

In [137]:
all_data = all_data.drop(["value","id","name"], axis=1)

# 전처리 2

In [158]:
all_data

,age,continent,contract_until,position,prefer_foot,reputation,stat_skill_moves,log_value,all_stat
0,31,south america,3,ST,left,5.0,4.0,18.520526,34.681064
1,27,europe,2,GK,right,4.0,1.0,18.092177,31.730070
2,31,south america,3,ST,right,5.0,3.0,18.197537,30.457970
3,32,europe,2,DF,right,4.0,3.0,17.747336,30.457970
4,25,europe,3,GK,right,3.0,1.0,18.035018,30.958422
...,...,...,...,...,...,...,...,...,...
12755,20,asia,3,MF,right,1.0,2.0,NaN,-20.532287
12756,18,europe,0,GK,right,1.0,1.0,NaN,-19.260187
12757,18,south america,3,MF,right,1.0,2.0,NaN,-19.260187
12758,18,europe,2,ST,right,1.0,2.0,NaN,-22.576035


In [165]:
dummy = pd.get_dummies(all_data[["continent","position","prefer_foot"]])
all_data = pd.concat([all_data,dummy],axis=1)
all_data.drop(["continent","position","prefer_foot"], axis=1, inplace=True)

# 데이터 나누기

In [167]:
num_train = len(train)
X = all_data_sprs[:num_train].toarray()
X_test = all_data_sprs[num_train:].toarray()
y = train["log_value"]

# 베이스라인 모델링

In [168]:
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

## XGB

In [169]:
xgb = XGBRegressor()
xgb.fit(X,y)
pred_xgb = xgb.predict(X_test)

## LGBM

In [170]:
lgb = LGBMRegressor()
lgb.fit(X,y)
pred_lgb = lgb.predict(X_test)

## GRB

In [171]:
gbr = GradientBoostingRegressor()
gbr.fit(X,y)
pred_gbr = gbr.predict(X_test)

## blend

In [172]:
pred = (pred_xgb+pred_lgb+pred_gbr)/3
pred = np.exp(pred)

# 제출

In [174]:
submission = pd.read_csv("data/submission.csv")
submission["value"]=pred
submission.to_csv("submission_try5.csv",index=False)

In [179]:
X_train = all_data[~all_data["log_value"].isnull()] 
X_test = all_data[all_data["log_value"].isnull()]

In [149]:
from pycaret.regression import *

In [180]:
data = setup(data=X_train, target="log_value", train_size=0.8)

,Description,Value
0,session_id,5056
1,Target,log_value
2,Original Data,"(8932, 17)"
3,Missing Values,False
4,Numeric Features,15
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(7145, 23)"


In [181]:
blender = blend_models(estimator_list=compare_models(n_select=3, sort="RMSE"))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0749,0.0115,0.1071,0.9947,0.0075,0.0055
1,0.0784,0.0126,0.1122,0.9934,0.0078,0.0058
2,0.0812,0.0137,0.1169,0.9926,0.0080,0.0060
3,0.0798,0.0131,0.1145,0.9936,0.0079,0.0059
4,0.0755,0.0110,0.1050,0.9943,0.0072,0.0055
5,0.0780,0.0121,0.1099,0.9941,0.0077,0.0058
6,0.0790,0.0135,0.1163,0.9930,0.0082,0.0059
7,0.0776,0.0125,0.1118,0.9935,0.0079,0.0058
8,0.0809,0.0138,0.1176,0.9924,0.0082,0.0060


In [182]:
final = finalize_model(blender)

In [186]:
X_test

,age,contract_until,reputation,stat_skill_moves,log_value,all_stat,continent_africa,continent_asia,continent_europe,continent_oceania,continent_south america,position_DF,position_GK,position_MF,position_ST,prefer_foot_left,prefer_foot_right
8932,33,4,5.0,5.0,NaN,34.681064,0,0,1,0,0,0,0,0,1,0,1
8933,26,4,5.0,5.0,NaN,32.501718,0,0,0,0,1,0,0,0,1,0,1
8934,27,5,4.0,4.0,NaN,31.094020,0,0,1,0,0,0,0,1,0,0,1
8935,27,2,4.0,4.0,NaN,30.457970,0,0,1,0,0,0,0,0,1,0,1
8936,32,2,4.0,4.0,NaN,30.457970,0,0,1,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12755,20,3,1.0,2.0,NaN,-20.532287,0,1,0,0,0,0,0,1,0,0,1
12756,18,0,1.0,1.0,NaN,-19.260187,0,0,1,0,0,0,1,0,0,0,1
12757,18,3,1.0,2.0,NaN,-19.260187,0,0,0,0,1,0,0,1,0,0,1
12758,18,2,1.0,2.0,NaN,-22.576035,0,0,1,0,0,0,0,0,1,0,1


In [187]:
X_test.drop("log_value",axis=1,inplace=True)

In [188]:
y_pred = predict_model(final, data=X_test)

In [189]:
y_pred

,age,contract_until,reputation,stat_skill_moves,all_stat,continent_africa,continent_asia,continent_europe,continent_oceania,continent_south america,position_DF,position_GK,position_MF,position_ST,prefer_foot_left,prefer_foot_right,Label
8932,33,4,5.0,5.0,34.681064,0,0,1,0,0,0,0,0,1,0,1,17.702382
8933,26,4,5.0,5.0,32.501718,0,0,0,0,1,0,0,0,1,0,1,18.273560
8934,27,5,4.0,4.0,31.094020,0,0,1,0,0,0,0,1,0,0,1,18.167544
8935,27,2,4.0,4.0,30.457970,0,0,1,0,0,0,0,0,1,0,1,18.204771
8936,32,2,4.0,4.0,30.457970,0,0,1,0,0,0,0,1,0,0,1,17.970356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12755,20,3,1.0,2.0,-20.532287,0,1,0,0,0,0,0,1,0,0,1,11.019494
12756,18,0,1.0,1.0,-19.260187,0,0,1,0,0,0,1,0,0,0,1,10.750158
12757,18,3,1.0,2.0,-19.260187,0,0,0,0,1,0,0,1,0,0,1,10.883722
12758,18,2,1.0,2.0,-22.576035,0,0,1,0,0,0,0,0,1,0,1,10.825050


In [190]:
y_pred = y_pred.reset_index()

In [191]:
pred = np.exp(y_pred["Label"])

In [192]:
pred

0       4.875812e+07
1       8.631897e+07
2       7.763615e+07
3       8.058080e+07
4       6.374213e+07
            ...     
3823    6.105278e+04
3824    4.663741e+04
3825    5.330160e+04
3826    5.026427e+04
3827    4.620024e+04
Name: Label, Length: 3828, dtype: float64

In [194]:
submission["value"]=pred

In [195]:
submission.to

,id,value
0,1,4.875812e+07
1,2,8.631897e+07
2,4,7.763615e+07
3,5,8.058080e+07
4,6,6.374213e+07
...,...,...
3823,16924,6.105278e+04
3824,16929,4.663741e+04
3825,16932,5.330160e+04
3826,16937,5.026427e+04


In [196]:
submission.to_csv("submission_try6.csv",index=False)